<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=127dcf5455f6a5a34d7aeb36153068e8dd401f860a4e18f7af955ac173e0899e
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-11 12:13:17
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.42 C
-------------------
Today PnL: 27.01 K (0.19%)
Current PnL: -17.65 L (-11.87%)
CY Booked + Current PnL: -6.62 L (-4.45%)
-------------------
Total profit:  2.08 L
Total loss:  -19.73 L
-------------------
Total Booked + Current PnL: 20.36 L (16.73%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.75 L (59.17%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.46%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.31,60.0,H-LC,11.20,173195.0,17018.0,9075.0,-0.71,10.90,5.24,16.71,22.0,1.88,1.26,30.23,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,60.0,M-LC,4.28,217949.0,16470.0,10745.0,0.52,8.17,4.93,13.51,60.0,1.53,1.59,21.80,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,107.68,65.0,M-SC,5.25,89460.0,-11317.0,11406.0,0.93,-11.23,12.75,0.09,245.0,-0.99,0.65,17.53,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.83,72.0,H-MC,3.53,114666.0,14162.0,16478.0,3.45,14.09,14.37,30.49,79.0,0.86,0.83,51.06,MH,ATH,METALS
41,ITC,452.00,-37.08,58.0,H-LC,3.00,203571.0,3433.0,17466.0,0.79,1.72,8.58,10.44,4.0,0.20,1.48,7.37,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,63.0,M-MC,6.99,226254.0,1292.0,166365.0,1.29,0.57,73.53,74.53,192.0,0.01,1.65,33.01,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-21.22,41.0,H-MC,1.98,135600.0,-188.0,42945.0,-0.37,-0.14,31.67,31.48,107.0,-0.00,0.99,11.96,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.94,57.0,H-MC,1.67,245731.0,-2000.0,77061.0,-0.01,-0.81,31.36,30.30,92.0,-0.03,1.79,12.38,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.68,53.0,H-MC,4.61,200655.0,-165.0,68945.0,0.92,-0.08,34.36,34.25,80.0,-0.00,1.46,5.11,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302731.0,-13111.0,125785.0,-0.27,-4.15,41.55,35.67,5.0,-0.10,2.20,9.13,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.47,44.0,H-LC,13.63,267718.0,353.0,65109.0,0.40,0.13,24.32,24.48,48.0,0.01,1.95,55.18,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.23,46.0,H-LC,8.17,224890.0,-9254.0,57887.0,0.31,-3.95,25.74,20.77,86.0,-0.16,1.64,2.69,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.16,230225.0,-11699.0,74961.0,0.17,-4.84,32.56,26.15,8.0,-0.16,1.68,8.79,X40,ATH,IT
56,RELIANCE,1533.00,-13.66,48.0,H-LC,6.30,215717.0,5351.0,23427.0,0.42,2.54,10.86,13.68,37.0,0.23,1.57,19.49,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-4.62,55.0,H-LC,1.27,236520.0,-9631.0,88837.0,-0.96,-3.91,37.56,32.18,16.0,-0.11,1.72,30.72,X200,ATH,IT
41,ITC,452.00,-37.08,58.0,H-LC,3.00,203571.0,3433.0,17466.0,0.79,1.72,8.58,10.44,4.0,0.20,1.48,7.37,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.08,63.0,H-LC,3.35,285971.0,20545.0,36233.0,0.38,7.74,12.67,21.40,11.0,0.57,2.08,16.13,XY25,NTT,FMCG
20,CIPLA,1795.00,-20.12,51.0,H-LC,5.94,213583.0,9083.0,32336.0,-0.31,4.44,15.14,20.25,10.0,0.28,1.55,13.56,X40N,BTT,PHARMA
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302731.0,-13111.0,125785.0,-0.27,-4.15,41.55,35.67,5.0,-0.10,2.20,9.13,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GREENPANEL,537.00,239.79,75.0,M-SC,6.32,151646.0,-31432.0,110413.0,13.91,-17.17,72.81,43.14,230.0,-0.28,1.10,42.64,XY24,NTT,MISC
49,NATIONALUM,247.44,-42.83,72.0,H-MC,3.53,114666.0,14162.0,16478.0,3.45,14.09,14.37,30.49,79.0,0.86,0.83,51.06,MH,ATH,METALS
57,REPCOHOME,880.00,-60.51,50.0,H-SC,7.96,178106.0,-57637.0,246926.0,2.08,-24.45,138.64,80.30,134.0,-0.23,1.30,20.57,XY24,NTT,FINANCE
32,HINDZINC,730.22,17.65,56.0,M-LC,1.79,191900.0,-13176.0,125752.0,1.92,-6.43,65.53,54.89,52.0,-0.10,1.40,16.12,X5K,ATH,METALS
9,AWL,485.00,-65.02,53.0,H-MC,5.11,249255.0,-52496.0,213911.0,1.85,-17.40,85.82,53.50,116.0,-0.25,1.81,10.71,XY24,NTT,FMCG


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ANGELONE,3033.0,3.57,34.0,H-SC,7.05,183713.0,-7293.0,64998.0,-4.12,-3.82,35.38,30.21,157.0,-0.11,1.34,16.27,X40N,NTT,FINANCE
82,VALIANTORG,838.0,-325.20,54.0,H-SC,8.41,125120.0,-38485.0,159803.0,-2.48,-23.52,127.72,74.15,139.0,-0.24,0.91,60.38,XR,NTT,CHEMICALS
67,SONACOMS,1006.0,-38.14,44.0,M-SC,11.97,76965.0,-24194.0,99085.0,-2.28,-23.92,128.74,74.03,202.0,-0.24,0.56,8.53,AR,BTT,AUTO
78,UJJIVANSFB,60.0,103.03,65.0,H-SC,12.70,127953.0,-14526.0,34048.0,-1.99,-10.20,26.61,13.70,163.0,-0.43,0.93,51.02,OX40N,NTT,BANKS
81,VAIBHAVGBL,521.0,52.62,60.0,H-SC,10.22,132094.0,-50681.0,161749.0,-1.84,-27.73,122.45,60.77,125.0,-0.31,0.96,18.31,XR,NTT,JEWELLERY


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,103.03,65.0,H-SC,12.70,127953.0,-14526.0,34048.0,-1.99,-10.20,26.61,13.70,163.0,-0.43,0.93,51.02,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,107.68,65.0,M-SC,5.25,89460.0,-11317.0,11406.0,0.93,-11.23,12.75,0.09,245.0,-0.99,0.65,17.53,OX40N,NTT,DURABLES
72,TATAELXSI,9161.0,-18.67,58.0,H-MC,8.02,110105.0,-21730.0,63949.0,-0.83,-16.48,58.08,32.03,98.0,-0.34,0.80,23.33,OX40N,NTT,IT
19,CERA,9475.0,-20.05,46.0,H-SC,2.23,145808.0,-30095.0,72117.0,-0.54,-17.11,49.46,23.89,149.0,-0.42,1.06,25.92,OX40N,NTT,CERAMICS
44,KANSAINER,340.0,-68.16,54.0,H-SC,2.33,219618.0,-50049.0,86376.0,-0.11,-18.56,39.33,13.47,138.0,-0.58,1.60,11.08,XY24,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,53.0,H-LC,13.71,292829.0,-53127.0,176986.0,0.17,-15.36,60.44,35.80,1.0,-0.30,2.13,3.74,X40,BTT,IT
39,INFY,2275.00,-17.44,53.0,H-LC,9.80,321886.0,8998.0,162681.0,-1.40,2.88,50.54,54.87,3.0,0.06,2.34,9.72,X40,BTT,IT
41,ITC,452.00,-37.08,58.0,H-LC,3.00,203571.0,3433.0,17466.0,0.79,1.72,8.58,10.44,4.0,0.20,1.48,7.37,X40,NTT,FMCG
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302731.0,-13111.0,125785.0,-0.27,-4.15,41.55,35.67,5.0,-0.10,2.20,9.13,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,53.0,H-LC,12.27,250637.0,-10982.0,130206.0,1.16,-4.20,51.95,45.57,7.0,-0.08,1.82,6.65,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,63.0,M-MC,6.99,226254.0,1292.0,166365.0,1.29,0.57,73.53,74.53,192.0,0.01,1.65,33.01,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.63,54.0,H-MC,3.75,189270.0,24600.0,43362.0,0.16,14.94,22.91,41.28,89.0,0.57,1.38,24.11,X40N,ATH,AC
84,VOLTAS,1918.49,-0.94,53.0,H-MC,3.17,207825.0,16083.0,79950.0,-0.67,8.39,38.47,50.08,99.0,0.20,1.51,15.76,XY25,ATH,AC
24,DABUR,735.00,-2.22,61.0,H-MC,6.01,212686.0,15658.0,73972.0,-0.39,7.95,34.78,45.49,102.0,0.21,1.55,22.02,XY24,BTT,FMCG
18,CAMS,5211.76,-3.61,56.0,H-SC,4.73,109536.0,7532.0,36388.0,0.54,7.38,33.22,43.06,122.0,0.21,0.80,25.76,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.83,72.0,H-MC,3.53,114666.0,14162.0,16478.0,3.45,14.09,14.37,30.49,79.0,0.86,0.83,51.06,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.03,57.0,M-SC,1.90,101392.0,9894.0,68855.0,-1.31,10.81,67.91,86.07,223.0,0.14,0.74,47.24,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.35,55.0,H-SC,12.18,138632.0,15296.0,118461.0,1.53,12.40,85.45,108.45,119.0,0.13,1.01,37.53,AR,ATH,MISC
86,WIPRO,420.00,-10.62,55.0,M-LC,6.28,156817.0,5872.0,103578.0,-1.10,3.89,66.05,72.51,53.0,0.06,1.14,10.15,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,66.0,H-MC,5.52,183160.0,3352.0,109053.0,1.36,1.86,59.54,62.52,88.0,0.03,1.33,36.59,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6700.00,51.0,L-SC,5.96,74640.0,-19170.0,95786.0,-0.05,-20.43,128.33,81.67,269.0,-0.20,0.54,44.54,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-3.83,48.0,H-SC,14.35,89569.0,-11394.0,105924.0,0.69,-11.29,118.26,93.63,148.0,-0.11,0.65,33.41,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,57.0,L-SC,43.43,88704.0,-24845.0,64896.0,-0.96,-21.88,73.16,35.27,268.0,-0.38,0.65,117.11,XR,NTT,HOTELS
48,MASFIN,398.61,-16.44,57.0,H-SC,13.07,94920.0,-3060.0,24660.0,0.64,-3.12,25.98,22.05,152.0,-0.12,0.69,37.97,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.83,72.0,H-MC,3.53,114666.0,14162.0,16478.0,3.45,14.09,14.37,30.49,79.0,0.86,0.83,51.06,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,57.0,L-SC,43.43,88704.0,-24845.0,64896.0,-0.96,-21.88,73.16,35.27,268.0,-0.38,0.65,117.11,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,3.01,82392.0,-19818.0,113207.0,0.42,-19.39,137.40,91.37,208.0,-0.18,0.60,33.75,XY25,NTT,FINANCE
75,TITAGARH,1548.00,-3.20,64.0,H-SC,6.48,158848.0,-33167.0,112051.0,-0.13,-17.27,70.54,41.08,158.0,-0.30,1.16,34.60,XY24,NTT,ENGINEERING
49,NATIONALUM,247.44,-42.83,72.0,H-MC,3.53,114666.0,14162.0,16478.0,3.45,14.09,14.37,30.49,79.0,0.86,0.83,51.06,MH,ATH,METALS
8,ATULAUTO,844.00,3608.11,58.0,M-SC,7.47,115640.0,-31728.0,83538.0,-1.30,-21.53,72.24,35.16,236.0,-0.38,0.84,18.11,XY24,NTT,AUTO


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.71
1,25,43.38
2,50,73.74


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.54
LC    34.38
MC    23.04
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.88
X40      15.25
X40N     12.99
XR       10.10
XY25      9.77
AR        8.61
OX40N     8.24
X200      1.72
MH        1.71
X5K       1.40
SR        1.29
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.41
H-LC    26.21
H-MC    20.10
M-SC    13.55
M-LC     7.11
M-MC     2.63
L-SC     1.58
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.36,-14.58,79.72
FMCG,11.94,-3.12,39.20
FINANCE,9.60,-16.71,62.42
BANKS,7.86,-11.07,62.21
MISC,6.09,-13.34,78.04
PAINTS,5.85,-14.76,40.45
ELECTRICAL,5.57,-7.45,66.67
AC,3.60,2.10,31.06
INSURANCE,3.38,-8.30,45.44


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2876142.0,22
AR,1313829.0,9
XR,1224635.0,13
X40,870599.0,10
X40N,627848.0,10
OX40N,514236.0,10
XY25,409486.0,7
SR,244588.0,2
X5K,125752.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3351470.0,28
H-LC,1389272.0,15
M-SC,1381731.0,17
H-MC,1190087.0,15
M-LC,403500.0,5
M-MC,316527.0,2
L-SC,244155.0,3
L-MC,59750.0,1
L-LC,38771.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1046206.0      6
M-SC       XY24       798309.0      7
H-SC       AR         783675.0      5
           XR         675712.0      6
H-LC       X40        581657.0      5
H-MC       XY24       486566.0      4
H-LC       AR         368926.0      2
M-MC       XY24       316527.0      2
H-SC       X40N       294217.0      4
           SR         244588.0      2
           OX40N      244239.0      4
H-LC       X40N       225083.0      4
H-MC       X40        218390.0      4
           XY25       187103.0      2
M-LC       XY24       163425.0      2
M-SC       AR         161228.0      2
L-SC       XR         160682.0      2
M-LC       X5K        125752.0      1
M-SC       OX40N      122575.0      4
           XR         115860.0      2
           XY25       113207.0      1
H-MC       XR         109053.0      1
           X40N       108548.0      2
M-LC       XR         103578.0      1
H-LC       X200        88837.0      1
L-SC       OX40N       83473.0      1
M-SC       X40         70552.0      1
H-LC       XY24        65109.0      1
H-MC       OX40N       63949.0      1
H-SC       MH          62833.0      1
L-MC       XR          59750.0      1
H-LC       XY25        59660.0      2
L-LC       XY25        38771.0      1
H-MC       MH          16478.0      1
M-LC       XY25        10745.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
